In [ ]:
using System; 
//#r "nuget: Plotly.NET.Interactive, 2.0.0-preview.1 " using Plotly.NET;

In [ ]:
public float IntensityRugulation(float ActiveEmotion, float avg)
{
    var tanh = (float)(((Math.Abs(ActiveEmotion / 2)) * (Math.Tanh((2 * avg - 100) / 40))) + (Math.Abs(ActiveEmotion / 2)));    
    var tanh2 = (float)((Math.Abs((ActiveEmotion / 2)) * (Math.Tanh(-(2 * avg - 100) / 40))) + (Math.Abs(ActiveEmotion / 2)));
    var newValue = (float)(tanh-(tanh2-1));
    return tanh;
}

In [ ]:
///// C = 20, E = 25, N = 15, O = 12, A = 30; // max=95
float C = 100, E =32, N = 15, O = 20, A = 30;
var ActiveEmotion = 7f;
var avg = (E + O) / 2;
Console.WriteLine("Averange: "+avg);
var ER = IntensityRugulation(ActiveEmotion,avg);

ER

Averange: 26


0.5822088

In [ ]:
public bool ResponseModulation(Name events)
{
    Console.WriteLine("\n---------------------Response Modulation------------------------");

    AppliedStrategy = false;
    
    //check personality
    var DAttentionDeployment = this.Personality.DStrategyAndPower.Where(
        (strategy, power) => strategy.Key == "Response Modulation");
    var ExistStrategy = DAttentionDeployment.Any();
    var StronglyStrategyPower = DAttentionDeployment.Select(p => p.Value.Trim() == "Strongly").FirstOrDefault();
    var ApplyStrategy = ExistStrategy && StronglyStrategyPower;

    //get event name and it construct the event
    var eventName = events.GetNTerm(3);
    var NewEvent = ReName(events);
    EventFatima = NewEvent.EventTypeFatima;
    var target = EventFatima.GetNTerm(4).ToString();

    Console.WriteLine("\nEvent name: " + eventName +
        "                          Target: " + target);

    //finds the appraisal value of event
    var Event = EventMatchingName(this.NewEA_character, eventName);

    if (ApplyStrategy && Event.AppraisalValue<0)
    {
        Console.WriteLine(" \n In progress...  ");
        Console.WriteLine(" Evaluating emotion intensity...  ");

        var CheckCharacter = this.NewAm_character.RecallAllEvents();
        var ActiveEmotion = this.NewEmotionalState_Character.GetAllEmotions().LastOrDefault();

        var avg = ((this.Personality.Extraversion + this.Personality.Openness) / 2);
        var tanh = (float)((Math.Abs((ActiveEmotion.Intensity / 2)) * (Math.Tanh(-(2 * avg - 100) / 40))) + (Math.Abs(ActiveEmotion.Intensity / 2)));
        //var gauss = -ActiveEmotion.Intensity * Math.Exp(-(Math.Pow((ERmood - 10), 2) / (10 * Math.Pow(Math.PI, 2)))) + ActiveEmotion.Intensity;
        var ModifiedValue = tanh;

        /*
        //crear nueva emoción a partir de la existente
        var IDemotion = this.NewEmotionalState_Character.GetAllEmotions().Select(e => e.CauseId).LastOrDefault();                
        var CausedEvent = this.NewEmotionalState_Character.GetAllEmotions().Select(e => e.EventName).LastOrDefault();
        var emotionType = this.NewEmotionalState_Character.GetEmotion(ActiveEmotion).EmotionType;
        var activeEmotion = this.NewEmotionalState_Character.GetEmotion(ActiveEmotion);
        */
        if (ModifiedValue > this.ObjetiveEmotion)
        {
            /*
            var NewEmotion = new EmotionalAppraisal.DTOs.EmotionDTO
            {

                Type = emotionType,
                Intensity = (float)ModifiedValue,
                CauseEventId = IDemotion,
                CauseEventName = CausedEvent.ToString(),
                Target = EventFatima.GetNTerm(3).ToString()

            };


            
            
            this.NewEmotionalState_Character.RemoveEmotion(activeEmotion, this.NewAm_character);
            this.NewEmotionalState_Character.AddActiveEmotion(NewEgeomotion, this.NewAm_character);


            var test = this.NewEmotionalState_Character.GetAllEmotions();
            
            var test2 = test.ToDto(this.NewAm_character);
            this.NewEmotionalState_Character.AddActiveEmotion(test2, this.NewAm_character);
            var test3 = this.NewEmotionalState_Character.GetAllEmotions().LastOrDefault();
            */

            

            
            //internal reappraisal
            var modifiedValue = (int)(Math.Round(ModifiedValue));
            var t = this.NewAm_character.Tick;
            
            var change = this.NewEmotionalState_Character.AddEmotion(
            ActiveEmotion, this.NewAm_character, new EmotionDispositionDTO() { Threshold = modifiedValue }, t);
            
            var activeEmotion2 = this.NewEmotionalState_Character.GetAllEmotions().LastOrDefault();
            var emoValence = this.NewEmotionalState_Character.GetAllEmotions().Select(e => (float)e.Valence).LastOrDefault();
            var emoIntencity = this.NewEmotionalState_Character.GetAllEmotions().Select(e => e.Intensity).LastOrDefault();
            var config = new EmotionalAppraisalConfiguration();
            
            var NewMood = ERmood + emoValence * (emoIntencity * config.EmotionInfluenceOnMoodFactor);
            var value = NewMood < -10 ? -10 : (NewMood > 10 ? 10 : NewMood);
            if (Math.Abs(value) < config.MinimumMoodValueForInfluencingEmotions)
                value = 0;

            this.NewEmotionalState_Character.Mood = value;
            
            AppliedStrategy = true;
        }
        else
        {

            Console.WriteLine("\n Strategy not applied due to : Emotion limit was achieved ===> " + (ModifiedValue >= this.ObjetiveEmotion));
            Console.WriteLine("\n New possible value = " + ModifiedValue + " -User defined limit = " + this.ObjetiveEmotion);
        }
    }
    else
    {
        Console.WriteLine("\n Strategy not applied due to :\n Agent's personality can apply the strategy ===> positive emotion = +" + Event.AppraisalValue);
    }
    Console.WriteLine("\n Response Modulation could be applied: " + AppliedStrategy);
    return AppliedStrategy;
}

In [ ]:

                                 
//testing emotion disposit
var t = this.NewAm_character.Tick;

List<int> vs = new()
{-12, -11, -10, -9, -8, -7, -6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12 };
foreach (var j in vs)
{
var testED = this.NewEmotionalState_Character.AddEmotion(
ActiveEmotion, NewAm_character, new EmotionDispositionDTO() { Threshold = j }, t);
if(testED != null)
    Console.WriteLine("index : "+j+"  level emotion: " + testED.Intensity);
}

In [ ]:
 public bool Test1(Name events)
        {
            Console.WriteLine("\n---------------------Response Modulation------------------------");

            AppliedStrategy = false;

            //check personality
            var DAttentionDeployment = this.Personality.DStrategyAndPower.Where(
                (strategy, power) => strategy.Key == "Response Modulation");
            var ExistStrategy = DAttentionDeployment.Any();
            var StronglyStrategyPower = DAttentionDeployment.Select(p => p.Value.Trim() == "Strongly").FirstOrDefault();
            var ApplyStrategy = ExistStrategy && StronglyStrategyPower;

            //get event name and it construct the event
            var eventName = events.GetNTerm(3);
            var NewEvent = ReName(events);
            EventFatima = NewEvent.EventTypeFatima;
            var target = EventFatima.GetNTerm(4).ToString();

            Console.WriteLine("\nEvent name: " + eventName +
                "                          Target: " + target);

            //finds the appraisal value of event
            var Event = EventMatchingName(this.NewEA_character, eventName);

            if (ApplyStrategy && Event.AppraisalValue < 0)
            {
                Console.WriteLine(" \n In progress...  ");
                Console.WriteLine(" Evaluating emotion intensity...  ");

                //emoción activa por el evento evaluado
                var ActiveEmotion = this.NewEmotionalState_Character.GetAllEmotions().LastOrDefault();

                //Calculo del nuevo nivel emocional
                var avg = ((this.Personality.Extraversion + this.Personality.Openness) / 2);
                var tanh = (float)((Math.Abs((ActiveEmotion.Intensity / 2)) * (Math.Tanh((2 * avg - 100) / 40))) + (Math.Abs(ActiveEmotion.Intensity / 2)));
                var ModifiedValue = tanh;

                if (ModifiedValue < this.ObjetiveEmotion)
                {
                    var tick = this.NewAm_character.Tick;
                    var emotionDispositionAgent = this.NewEA_character.GetEmotionDisposition(ActiveEmotion.EmotionType);

                    var NewEmotion = new EmotionalAppraisal.DTOs.EmotionDTO
                    {
                        Type = ActiveEmotion.EmotionType,
                        Intensity = 0.7f,
                        CauseEventName = events.GetNTerm(3).ToString(),
                        Target = EventFatima.GetNTerm(4).ToString()
                    };

                    var es = new ConcreteEmotionalState();
                    es.AddActiveEmotion(NewEmotion, this.NewAm_character);

                    var NewActiveEmotion = es.GetAllEmotions().LastOrDefault();

                    var NewActiveIEmotion = NewActiveEmotion as IEmotion;

                    //this.NewEmotionalState_Character.Mood = ERmood;
                    this.NewEmotionalState_Character.RemoveEmotion(ActiveEmotion, this.NewAm_character);
                    var change = this.NewEmotionalState_Character.AddEmotion(
                    NewActiveIEmotion, this.NewAm_character, emotionDispositionAgent, tick);

                    AppliedStrategy = true;
                }
                else
                {
                    Console.WriteLine("\n+ Strategy not applied due to : Emotion limit was achieved ===> " + (ModifiedValue >= this.ObjetiveEmotion));
                    Console.WriteLine("\n New possible value = " + ModifiedValue + " -User defined limit = " + this.ObjetiveEmotion);
                }
            }
            else
            {
                Console.WriteLine("\n Strategy not applied due to :\n Agent's personality can apply the strategy ===> positive emotion = " + Event.AppraisalValue);
            }
            Console.WriteLine("\n Response Modulation could be applied: " + AppliedStrategy);
            return AppliedStrategy;
        }